In [1]:
from pringles.simulator import Simulator,Simulation, Event
from pringles.models import Coupled
from pringles.utils import VirtualTime
mySimulator = Simulator(cdpp_bin_path='bin/', user_models_dir='src/')

In [2]:
for atomic in mySimulator.atomic_registry.discovered_atomics:
    if atomic.__name__ == "Server":
        Server = atomic
    elif atomic.__name__ == "AutoScaler":
        AutoScaler = atomic
    elif atomic.__name__ == "ServerQueue":
        ServerQueue = atomic
    elif atomic.__name__ == "Dispatcher":
        Dispatcher = atomic
    elif atomic.__name__ == "Generator":
        Generator = atomic    

In [3]:
number_of_servers = 2

autoScalerDefaultConfig = {
    'numberOfServers': number_of_servers,
    'exponentialWeight' : 0.6,
    'loadLowerBound': 0.3,
    'loadUpperBound': 0.8,
    'loadUpdatesToBreakIdle': 10,
    'server0': 'free',
}

dispatcherDefaultConfig = {
    'numberOfServers': number_of_servers,
    'server0': 'free'
}

serverDefaultConfig = {
    'distribution': 'exponential',
    'mean': 1.0,
    'setupTime': '00:00:10:00'
}

serverOnDefaultConifg = {
    'initialStatus': 'free',
    **serverDefaultConfig
}

serverOffDefaultConifg = {
    'initialStatus': 'off',
    **serverDefaultConfig
}

serverQueueDefaultConfig = {
    'size': 100,
    'currentSizeFrequency': '00:00:05:00'
}

normalTrafficGenDefaultConfig = {
    'distribution': 'normal',
    'mean': 0.01,
    'deviation': 1,
    'initial': 0,
    'increment': 1
}

In [4]:
scaler = AutoScaler('as', **autoScalerDefaultConfig)
dispatcher = Dispatcher('dispatcher', **dispatcherDefaultConfig)
queue = ServerQueue('queue', **serverQueueDefaultConfig)
server0 = Server('server0', **serverOnDefaultConifg)
server1 = Server('server1', **serverOffDefaultConifg)
trafficGenerator = Generator('gen', **normalTrafficGenDefaultConfig)

In [5]:
print("--Dispatcher ports")
print("Inport names: ", [port.name for port in dispatcher.inports])
print("Outport names: ", [port.name for port in dispatcher.outports])

print("--Server ports--")
print("Inport names: ", [port.name for port in server0.inports])
print("Outport names: ", [port.name for port in server0.outports])

print("--Generator ports--")
print("Inport names: ", [port.name for port in trafficGenerator.inports])
print("Outport names: ", [port.name for port in trafficGenerator.outports])

print("--Queue ports--")
print("Inport names: ", [port.name for port in queue.inports])
print("Outport names: ", [port.name for port in queue.outports])

print("--Scaler ports--")
print("Inport names: ", [port.name for port in scaler.inports])
print("Outport names: ", [port.name for port in scaler.outports])


--Dispatcher ports
Inport names:  ['newJob', 'jobDone', 'serverStackInfo']
Outport names:  ['requestJob', 'server0', 'server1', 'server2']
--Server ports--
Inport names:  ['job', 'powerSignal']
Outport names:  ['done', 'ready']
--Generator ports--
Inport names:  ['stop']
Outport names:  ['out']
--Queue ports--
Inport names:  ['in', 'emit']
Outport names:  ['out', 'discarded', 'current_size']
--Scaler ports--
Inport names:  ['queueLoad', 'serverResponse']
Outport names:  ['serverStatus', 'server0', 'server1']


In [10]:
subcomponents = [trafficGenerator, queue, dispatcher, scaler, server0, server1]

top_model = (
    Coupled(name='top', subcomponents=subcomponents)
        .add_outport('processedJob')
        .add_outport('droppedJob')
        .add_coupling(trafficGenerator.get_port('out'), queue.get_port('in'))
        .add_coupling(queue.get_port('out'), dispatcher.get_port('newJob'))
        .add_coupling(queue.get_port('current_size'), scaler.get_port('queueLoad'))
        .add_coupling(queue.get_port('discarded'), 'droppedJob')
        .add_coupling(dispatcher.get_port('requestJob'), queue.get_port('emit'))
        .add_coupling(dispatcher.get_port('server0'), server0.get_port('job'))
        .add_coupling(dispatcher.get_port('server1'), server1.get_port('job'))
        .add_coupling(scaler.get_port('server0'), server0.get_port('powerSignal'))
        .add_coupling(scaler.get_port('server1'), server1.get_port('powerSignal'))
        .add_coupling(scaler.get_port('serverStatus'), dispatcher.get_port('serverStackInfo'))
        .add_coupling(server0.get_port('done'), dispatcher.get_port('jobDone'))
        .add_coupling(server1.get_port('done'), dispatcher.get_port('jobDone'))
        .add_coupling(server0.get_port('ready'), scaler.get_port('serverResponse'))
        .add_coupling(server1.get_port('ready'), scaler.get_port('serverResponse'))
        .add_coupling(server0.get_port('done'), 'processedJob')
        .add_coupling(server1.get_port('done'), 'processedJob')
)

top_model

In [7]:
a_simulation = Simulation(
    top_model = top_model, 
    duration = VirtualTime.of_minutes(60), 
    working_dir='sim_results/'
)

results = mySimulator.run_simulation(a_simulation)

In [14]:
df = results.logs_dfs['as']
df[df['port']=='queueload']

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:00:000,top(07),queueload,0.01,as(04)
1,0,L,X,00:00:05:000,top(07),queueload,0.06,as(04)
2,0,L,X,00:00:10:000,top(07),queueload,0.05,as(04)
3,0,L,X,00:00:15:000,top(07),queueload,0.03,as(04)
4,0,L,X,00:00:20:000,top(07),queueload,0.02,as(04)
5,0,L,X,00:00:25:000,top(07),queueload,0.04,as(04)
6,0,L,X,00:00:30:000,top(07),queueload,0.08,as(04)
7,0,L,X,00:00:35:000,top(07),queueload,0.06,as(04)
8,0,L,X,00:00:40:000,top(07),queueload,0.04,as(04)
9,0,L,X,00:00:45:000,top(07),queueload,0.06,as(04)
